## Preprocess Data/ Setup Keyspace

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating a list of filepaths to all csv files under the event_data folder

In [ ]:
# Check current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# Check the number of rows in the output CSV file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

#### Creating a Cluster

In [ ]:
# Make a connection to a Cassandra instance on the local machine at (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songplay
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor' : 1}"""
    )
except Exception as ex:
    print(ex)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('songplay')
except Exception as e:
    print(e)

### Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## QUERY 1: This query will retrieve all of the rows in the table that belong to session_id 338 and item_in_session 4. It will print out the name of the artist, name of the song, and length of the song that was listened to by that artist in that session and item in session. You can use this query to find details about the artist and song that was listened to in a particular session and item within session. The WHERE clause within this query can be modified to search for other session and item within session ID's.

### The combination of session_id and item_in_session is globally unique, which makes it a good composite primary key. Every user's session has a unique session ID, and every song listened to during that session has a unique item_in_session ID.

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_session"
query = query + "(session_id INT, item_in_session INT, artist TEXT, song_title TEXT, \
                  song_length FLOAT, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query = "SELECT * from music_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

In [ ]:
## Run the actual query

query =  "SELECT artist, song_title, song_length FROM music_session \
                WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.song_length)

ANSWER TO QUERY: This shows that the song 'Music Matters (Mark Knight Dub)' sung by the artist 'Faithless' and having a song length of 495 seconds was the song that was within the session with session ID 338 and item in session 4. 

## QUERY 2: This query will retrieve all of the rows in the table that belong to user_id 10 and session_id 182. It will print out the name of the artist, name of the song, and the first and last name of the user that has that user_id. You can use this query to find details about the artist and song that was listened to by a particular user_id and within a particular session (denoted by the session_id). The WHERE clause within this query can be modified to search for other users and sessions. The results are sorted according to the item_in_session, which acts as a clustering column as it is a part of the primary key but outside the partition key.

### Each user can have multiple sessions in which they listened to a song. The combination of user_id, session_id, and item_in_session is globally unique, which makes it a good combination for a primary key. We need to include the user_id in the primary key of this table since we are querying first for a particular user with a user_id of 10. The user_id needs to be before the session_id since we are querying first for the user and then for a session_id of 182 for that user. The item_in_session is a clustering column because it is used to purely order the data within the partition, and the partition is defined by every unique (user_id, session_id) value. All unique (user_id, session_id) records will exist on one node within the Cassandra cluster, and the records on that node will be sorted by their item_in_session values.

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) 
# and user (first and last name) for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song_title TEXT, \
                  first_name TEXT, last_name TEXT, PRIMARY KEY ((user_id, session_id), \
                  item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (user_id, session_id, item_in_session, artist, song_title, \
                                           first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
query = "SELECT * from user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

In [ ]:
## Run the actual query

query =  "SELECT artist, song_title, first_name, last_name FROM user_session \
                WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

ANSWER TO QUERY: This shows that a user of the music app named Sylvie Cruz listened to four songs, each by a different artist, in session_id 182. The names of the four songs are 'Keep On Keepin' On', 'Greece 2000', 'Kilometer', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)'.

## QUERY 3: This query will retrieve all of the users that listened to a particular song. More specifically, it will print out the the first and last name of the user that listening to a song with the name 'All Hands Against His Own'. You can use this query to generate a list of the first and last names of all users that listened to a particular song by modifying the WHERE clause as needed. 

### NOTE: The combination of song_title and user_id is actually NOT globally unique, since there are several records in which the same users listened to the same songs multiple times, which means that there are multiple rows with the same song_title and user_id. However, for the purposes of this query, using song_title and user_id as a primary key will yield the same result as using a truly unique primary key, since we are only interested in finding out who all listened to only one particular song. Even if there are users who listened to a song multiple times, when we try to load in all of the data from the event_datafile_new.csv file, Cassandra will reject all duplicates by default since the primary key is on the song_title and user_id. Only the first unique (song_title, user_id) combination will be kept when inserting data into the table and this will still tell us the name of the user that listened to that song. In short, Cassandra will keep one record (a single listening session) and reject all other duplicate records (all other listening sessions in which the same user listened to the same song, if any). For the purposes of this query, this will yield the correct result since we are interested in getting a list of DIFFERENT users who listened to the same song ('All Hands Against His Own').

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the 
# song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_song_history"
query = query + "(song_title TEXT, user_id INT, first_name TEXT, last_name TEXT, \
                  PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_history (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
query = "SELECT * from user_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

In [ ]:
## Run the actual query

query =  "SELECT first_name, last_name FROM user_song_history \
                WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

In [ ]:
for row in session.execute("SELECT COUNT(*) FROM user_song_history"):
    print(row)

ANSWER TO QUERY: This shows that there were three total users in the music app history who listened to the song 'All Hands Against His Own'. These users are Jacqueline Lynch, Tegan Levine, and Sara Johnson.

## Clean up keyspace

### Drop the tables before closing out the sessions

In [ ]:
## Drop all tables before closing out the session

query = "DROP TABLE music_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()